In [160]:
pip install tld 

Note: you may need to restart the kernel to use updated packages.


In [161]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import bert 


In [162]:
df = pd.read_csv('url.csv')
df.head()

,id,dateadded,url,url_status,last_online,threat,tags,urlhaus_link,reporter
0,2271848,8/11/2022 19:13,http://223.151.251.100:52112/.i,online,None,malware_download,hajime,https://urlhaus.abuse.ch/url/2271848/,geenensp
1,2271847,8/11/2022 18:59,http://180.177.215.49:1695/.i,online,None,malware_download,hajime,https://urlhaus.abuse.ch/url/2271847/,geenensp
2,2271846,8/11/2022 18:52,http://125.229.120.204:23742/.i,online,None,malware_download,hajime,https://urlhaus.abuse.ch/url/2271846/,geenensp
3,2271845,8/11/2022 18:36,http://213.222.137.243:32815/.i,online,None,malware_download,hajime,https://urlhaus.abuse.ch/url/2271845/,geenensp
4,2271844,8/11/2022 18:23,http://171.232.110.180:49532/.i,online,None,malware_download,hajime,https://urlhaus.abuse.ch/url/2271844/,geenensp


#### hajime Hajime is a malware which appears to be similar to the Wifatch malware in that it appears to attempt to secure devices.

In [163]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79337 entries, 0 to 79336
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            79337 non-null  int64 
 1   dateadded     79337 non-null  object
 2   url           79337 non-null  object
 3   url_status    79337 non-null  object
 4   last_online   79337 non-null  object
 5   threat        79337 non-null  object
 6   tags          79337 non-null  object
 7   urlhaus_link  79337 non-null  object
 8   reporter      79337 non-null  object
dtypes: int64(1), object(8)
memory usage: 5.4+ MB


In [164]:
#drop unnecessary columns
df.drop(['url_status', 'reporter', 'id', 'dateadded', 'last_online',
        'urlhaus_link'], axis=1, inplace=True)
df.head()


,url,threat,tags
0,http://223.151.251.100:52112/.i,malware_download,hajime
1,http://180.177.215.49:1695/.i,malware_download,hajime
2,http://125.229.120.204:23742/.i,malware_download,hajime
3,http://213.222.137.243:32815/.i,malware_download,hajime
4,http://171.232.110.180:49532/.i,malware_download,hajime


In [165]:
df.isnull().sum()


url       0
threat    0
tags      0
dtype: int64

In [166]:
df.head()

,url,threat,tags
0,http://223.151.251.100:52112/.i,malware_download,hajime
1,http://180.177.215.49:1695/.i,malware_download,hajime
2,http://125.229.120.204:23742/.i,malware_download,hajime
3,http://213.222.137.243:32815/.i,malware_download,hajime
4,http://171.232.110.180:49532/.i,malware_download,hajime


In [167]:
# All index of tags 
df.tags.value_counts()

elf,Mozi                                    17220
elf,mirai,Mozi                              11493
32-bit,elf,mips,Mozi                        10237
hajime                                       7329
32-bit,arm,elf,mirai,Mozi                    4170
                                            ...  
exe,Neshta                                      1
encrypted,fake-jpg,RedLineStealer               1
BetaBot,Neurevt,zip                             1
exe,RedLine,RedLineStealer,XFilesStealer        1
Pony                                            1
Name: tags, Length: 1092, dtype: int64

In [168]:
# count number of exe ,None, hajime, elf,Mozi
df.tags.value_counts()[['exe', 'None', 'hajime', 'elf',
                        'Mozi', '32-bit,arm,elf,mirai,Mozi', '32-bit,elf,mips,Mozi']]


exe                            455
None                          2855
hajime                        7329
elf                            785
Mozi                          3842
32-bit,arm,elf,mirai,Mozi     4170
32-bit,elf,mips,Mozi         10237
Name: tags, dtype: int64

In [169]:
df.groupby('tags').count()

,url,threat
tags,,
",32-bit,arm,elf",24,24
",32-bit,arm,elf,mirai",1,1
",32-bit,elf,gafgyt,mips",1,1
",32-bit,elf,mips",357,357
",32-bit,elf,mips,mirai",16,16
...,...,...
vbs,14,14
vjw0rm,3,3
xlsx,1,1


In [170]:
df.groupby('tags').describe()

url         \
                        count unique   
tags                                   
,32-bit,arm,elf            24     24   
,32-bit,arm,elf,mirai       1      1   
,32-bit,elf,gafgyt,mips     1      1   
,32-bit,elf,mips          357    357   
,32-bit,elf,mips,mirai     16     16   
...                       ...    ...   
vbs                        14     14   
vjw0rm                      3      3   
xlsx                        1      1   
younglotus                  2      2   
zip                        24     24   

                                                                            \
                                                                       top   
tags                                                                         
,32-bit,arm,elf                        http://221.214.159.194:43444/bin.sh   
,32-bit,arm,elf,mirai                     http://46.23.109.47/snickers/arm   
,32-bit,elf,gafgyt,mips                       http://209.141.41.137/s.mips   
,32-bit,elf,mips                          http://61.53.91.181:54907/bin.sh   
,32-bit,elf,mips,mirai                      http://112.248.244.224:58526/i   
...                                                                    ...   
vbs                      http://gotovacoil.com/goes/Protected%20Client.vbs   
vjw0rm                              http://109.206.241.81/htdocs/tSLKM.exe   
xlsx                     http://bookstore.neu.edu.tr/KGB%20Numaralari%2...   
younglotus                                     http://120.48.11.44/zxc.exe   
zip                      https://login.yoursecurecloud.de/seafhttp/file...   

                             threat                                
                        freq  count unique               top freq  
tags                                                               
,32-bit,arm,elf            1     24      1  malware_download   24  
,32-bit,arm,elf,mirai      1      1      1  malware_download    1  
,32-bit,elf,gafgyt,mips    1      1      1  malware_download    1  
,32-bit,elf,mips           1    357      1  malware_download  357  
,32-bit,elf,mips,mirai     1     16      1  malware_download   16  
...                      ...    ...    ...               ...  ...  
vbs                        1     14      1  malware_download   14  
vjw0rm                     1      3      1  malware_download    3  
xlsx                       1      1      1  malware_download    1  
younglotus                 1      2      1  malware_download    2  
zip                        1     24      1  malware_download   24  

[1092 rows x 8 columns]

In [174]:

plt.figure(figsize=(14, 10))


sns.barplot(
    x="total_malware",
    y="Malware_url",
    data=df,
    estimator=sum,
    ci=None,
    order=["exe", "None", "hajime", "elf", "Mozi", "32-bit,arm,elf,mirai,Mozi", "32-bit,elf,mips,Mozi"],
    color='#69b3a2'
)


ValueError: Could not interpret input 'total_malware'

<Figure size 1008x720 with 0 Axes>